<a href="https://colab.research.google.com/github/arfanrifqi/loan_prediction/blob/main/loan_data_prediction(preprocessing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#library

In [288]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#load dataset

In [289]:
load_data = pd.read_csv('/content/gdrive/MyDrive/dataset/loan_data_2007_2014.csv')

load_data = load_data[['loan_amnt','funded_amnt',
       'funded_amnt_inv','term','installment','grade',
       'emp_title','emp_length','home_ownership','annual_inc',
       'verification_status','issue_d','loan_status','pymnt_plan',
       'purpose','title','addr_state','dti','delinq_2yrs','earliest_cr_line',
       'inq_last_6mths','open_acc','pub_rec','revol_bal','revol_util','total_acc',
       'tot_cur_bal','tot_coll_amt','total_rev_hi_lim','next_pymnt_d','mths_since_last_delinq',
       'initial_list_status','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int',
       'total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt',
       'last_credit_pull_d','collections_12_mths_ex_med','policy_code','application_type','acc_now_delinq']].copy()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# determine target data

In [290]:
load_data['term_months'] = load_data['term'].replace([' 36 months',' 60 months'],['36','60'])
term_num = ['term_months']

load_data[term_num] = load_data[term_num].apply(pd.to_numeric, errors='coerce', axis=1)
print(load_data['term_months'].dtypes)

int64


In [291]:
load_data['verification_status'].replace({'Source Verified':'Verified'})

0             Verified
1             Verified
2         Not Verified
3             Verified
4             Verified
              ...     
466280        Verified
466281        Verified
466282        Verified
466283        Verified
466284        Verified
Name: verification_status, Length: 466285, dtype: object

In [292]:
bad_status = [
    'Charged Off' 
    , 'Default'
    , 'Does not meet the credit policy. Status:Charged Off'
    , 'Late (31-120 days)'
]
load_data['bad_flag'] = np.where(load_data['loan_status'].isin(bad_status), 1, 0)

In [293]:
load_data.drop('loan_status', axis=1, inplace=True)
load_data.head(1)

,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,tot_cur_bal,tot_coll_amt,total_rev_hi_lim,next_pymnt_d,mths_since_last_delinq,initial_list_status,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,term_months,bad_flag
0,5000,5000,4975.0,36 months,162.87,B,NaN,10+ years,RENT,24000.0,Verified,Dec-11,n,credit_card,Computer,AZ,27.65,0.0,Jan-85,1.0,3.0,0.0,13648,83.7,9.0,NaN,NaN,NaN,NaN,NaN,f,5861.071414,5831.78,5000.0,861.07,0.0,0.0,0.0,Jan-15,171.62,Jan-16,0.0,1,INDIVIDUAL,0.0,36,0


#CLEANING, PREPROCESSING, FEATURE ENGINEERING

## emp_length columns

In [294]:
load_data['emp_length_int'] = load_data['emp_length'].str.replace('\+ years', '')
load_data['emp_length_int'] = load_data['emp_length_int'].str.replace('< 1 year', str(0))
load_data['emp_length_int'] = load_data['emp_length_int'].str.replace(' years', '')
load_data['emp_length_int'] = load_data['emp_length_int'].str.replace(' year', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [295]:
load_data['emp_length_int'] = load_data['emp_length_int'].astype(float)

In [296]:
load_data.drop('emp_length', axis=1, inplace=True)

## earliest_cr_line columns

In [297]:
load_data['earliest_cr_line_date'] = pd.to_datetime(load_data['earliest_cr_line'], format='%b-%y')
load_data['earliest_cr_line_date'].head()

0   1985-01-01
1   1999-04-01
2   2001-11-01
3   1996-02-01
4   1996-01-01
Name: earliest_cr_line_date, dtype: datetime64[ns]

In [298]:
load_data['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - load_data['earliest_cr_line_date']) / np.timedelta64(1, 'M')))
load_data['mths_since_earliest_cr_line'].head(3)

0    395.0
1    224.0
2    193.0
Name: mths_since_earliest_cr_line, dtype: float64

In [299]:
load_data.loc[load_data['mths_since_earliest_cr_line']<0, 'mths_since_earliest_cr_line'] = load_data['mths_since_earliest_cr_line'].max()

In [300]:
load_data.drop(['earliest_cr_line', 'earliest_cr_line_date'], axis=1, inplace=True)

## issue_d columns

In [301]:
load_data['issue_d_date'] = pd.to_datetime(load_data['issue_d'], format='%b-%y')
load_data['mths_since_issue_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - load_data['issue_d_date']) / np.timedelta64(1, 'M')))

In [302]:
load_data.drop(['issue_d', 'issue_d_date'], axis=1, inplace=True)

## last_pymnt_d columns

In [303]:
load_data['last_pymnt_d_date'] = pd.to_datetime(load_data['last_pymnt_d'], format='%b-%y')
load_data['mths_since_last_pymnt_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - load_data['last_pymnt_d_date']) / np.timedelta64(1, 'M')))

In [304]:
load_data.drop(['last_pymnt_d', 'last_pymnt_d_date'], axis=1, inplace=True)

## next_pymnt_d columns

In [305]:
load_data['next_pymnt_d_date'] = pd.to_datetime(load_data['next_pymnt_d'], format='%b-%y')
load_data['mths_since_next_pymnt_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - load_data['next_pymnt_d_date']) / np.timedelta64(1, 'M')))

In [306]:
load_data.drop(['next_pymnt_d', 'next_pymnt_d_date'], axis=1, inplace=True)

## last_credit_pull_d columns

In [307]:
load_data['last_credit_pull_d_date'] = pd.to_datetime(load_data['last_credit_pull_d'], format='%b-%y')
load_data['mths_since_last_credit_pull_d'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - load_data['last_credit_pull_d_date']) / np.timedelta64(1, 'M')))

In [308]:
load_data.drop(['last_credit_pull_d', 'last_credit_pull_d_date'], axis=1, inplace=True)

# EXPLORATORY DATA ANALYSIS

## correlation numerical features

In [309]:
corr_with_target = load_data.corrwith(load_data.bad_flag).sort_values(ascending = False).abs().to_frame('correlation_with_target').reset_index().head(36)
unique_values = load_data.nunique().to_frame('unique_values').reset_index()
corr_with_unique = pd.merge(corr_with_target, unique_values, on = 'index', how = 'inner')
corr_with_unique

,index,correlation_with_target,unique_values
0,bad_flag,1.000000,2
1,recoveries,0.441171,22773
2,collection_recovery_fee,0.299227,20275
3,mths_since_last_pymnt_d,0.244581,98
4,mths_since_next_pymnt_d,0.154003,100
5,total_rec_late_fee,0.147750,5808
6,mths_since_last_credit_pull_d,0.136595,103
7,mths_since_issue_d,0.087575,91
8,inq_last_6mths,0.073102,28
9,term_months,0.063565,2


## correlation categorical features

In [310]:
load_data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,grade,emp_title,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,tot_cur_bal,tot_coll_amt,total_rev_hi_lim,mths_since_last_delinq,initial_list_status,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,term_months,bad_flag,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,mths_since_last_pymnt_d,mths_since_next_pymnt_d,mths_since_last_credit_pull_d
0,5000,5000,4975.0,36 months,162.87,B,NaN,RENT,24000.0,Verified,n,credit_card,Computer,AZ,27.65,0.0,1.0,3.0,0.0,13648,83.7,9.0,NaN,NaN,NaN,NaN,f,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,171.62,0.0,1,INDIVIDUAL,0.0,36,0,10.0,395.0,72.0,35.0,NaN,23.0
1,2500,2500,2500.0,60 months,59.83,C,Ryder,RENT,30000.0,Source Verified,n,car,bike,GA,1.00,0.0,5.0,3.0,0.0,1687,9.4,4.0,NaN,NaN,NaN,NaN,f,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,119.66,0.0,1,INDIVIDUAL,0.0,60,1,0.0,224.0,72.0,56.0,NaN,51.0
2,2400,2400,2400.0,36 months,84.33,C,NaN,RENT,12252.0,Not Verified,n,small_business,real estate business,IL,8.72,0.0,2.0,2.0,0.0,2956,98.5,10.0,NaN,NaN,NaN,NaN,f,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,649.91,0.0,1,INDIVIDUAL,0.0,36,0,10.0,193.0,72.0,42.0,NaN,23.0
3,10000,10000,10000.0,36 months,339.31,C,AIR RESOURCES BOARD,RENT,49200.0,Source Verified,n,other,personel,CA,20.00,0.0,1.0,10.0,0.0,5598,21.0,37.0,NaN,NaN,NaN,35.0,f,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,357.48,0.0,1,INDIVIDUAL,0.0,36,0,10.0,262.0,72.0,35.0,NaN,35.0
4,3000,3000,3000.0,60 months,67.79,B,University Medical Group,RENT,80000.0,Source Verified,n,other,Personal,OR,17.94,0.0,0.0,15.0,0.0,27783,53.9,38.0,NaN,NaN,NaN,38.0,f,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,67.79,0.0,1,INDIVIDUAL,0.0,60,0,1.0,263.0,72.0,23.0,22.0,23.0


In [311]:
from scipy.stats import chi2_contingency

In [312]:
cat_features = ['grade','emp_title','home_ownership','verification_status','pymnt_plan',
                'purpose','title','addr_state','initial_list_status','application_type']

error = 0.05

for column in cat_features:
    table = load_data[[column, 'bad_flag']].pivot_table(index='bad_flag',
                                                   columns=column,
                                                   aggfunc=lambda x: len(x))
    _, p_value, _, _ = chi2_contingency(table)

    print('Variable:', column)
    print('p-value: ', p_value)
    if p_value < error:
        print('Variabel ' + column + ' dan bad_flag berkorelasi')
    else:
        print('Variabel ' + column + ' dan bad_flag tidak berkorelasi')
    print('')

Variable: grade
p-value:  0.0
Variabel grade dan bad_flag berkorelasi

Variable: emp_title
p-value:  nan
Variabel emp_title dan bad_flag tidak berkorelasi

Variable: home_ownership
p-value:  nan
Variabel home_ownership dan bad_flag tidak berkorelasi

Variable: verification_status
p-value:  1.1011712895478403e-208
Variabel verification_status dan bad_flag berkorelasi

Variable: pymnt_plan
p-value:  0.007185645086446904
Variabel pymnt_plan dan bad_flag berkorelasi

Variable: purpose
p-value:  0.0
Variabel purpose dan bad_flag berkorelasi

Variable: title
p-value:  nan
Variabel title dan bad_flag tidak berkorelasi

Variable: addr_state
p-value:  nan
Variabel addr_state dan bad_flag tidak berkorelasi

Variable: initial_list_status
p-value:  7.216184133923969e-236
Variabel initial_list_status dan bad_flag berkorelasi

Variable: application_type
p-value:  1.0
Variabel application_type dan bad_flag tidak berkorelasi



In [313]:
load_data = load_data.drop(['application_type','addr_state','title','home_ownership','emp_title','term'], axis=1)
load_data.head(2)

,loan_amnt,funded_amnt,funded_amnt_inv,installment,grade,annual_inc,verification_status,pymnt_plan,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,tot_cur_bal,tot_coll_amt,total_rev_hi_lim,mths_since_last_delinq,initial_list_status,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,term_months,bad_flag,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,mths_since_last_pymnt_d,mths_since_next_pymnt_d,mths_since_last_credit_pull_d
0,5000,5000,4975.0,162.87,B,24000.0,Verified,n,credit_card,27.65,0.0,1.0,3.0,0.0,13648,83.7,9.0,NaN,NaN,NaN,NaN,f,5861.071414,5831.78,5000.00,861.07,0.0,0.00,0.00,171.62,0.0,1,0.0,36,0,10.0,395.0,72.0,35.0,NaN,23.0
1,2500,2500,2500.0,59.83,C,30000.0,Source Verified,n,car,1.00,0.0,5.0,3.0,0.0,1687,9.4,4.0,NaN,NaN,NaN,NaN,f,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,119.66,0.0,1,0.0,60,1,0.0,224.0,72.0,56.0,NaN,51.0


In [314]:
load_data = load_data.drop(['policy_code','pymnt_plan'], axis=1)

# HANDLE MISSING VALUES

In [315]:
print('Status Missing Values :',load_data.isnull().values.any())
print('\nJumlah Missing Values masing-masing kolom, adalah:')
print(load_data.isnull().sum().sort_values(ascending=False))

Status Missing Values : True

Jumlah Missing Values masing-masing kolom, adalah:
mths_since_last_delinq           250351
mths_since_next_pymnt_d          227214
total_rev_hi_lim                  70276
tot_coll_amt                      70276
tot_cur_bal                       70276
emp_length_int                    21008
mths_since_last_pymnt_d             376
revol_util                          340
collections_12_mths_ex_med          145
mths_since_last_credit_pull_d        42
inq_last_6mths                       29
pub_rec                              29
delinq_2yrs                          29
total_acc                            29
mths_since_earliest_cr_line          29
acc_now_delinq                       29
open_acc                             29
annual_inc                            4
collection_recovery_fee               0
funded_amnt_inv                       0
installment                           0
mths_since_issue_d                    0
grade                                 0

In [316]:
load_data.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'grade',
       'annual_inc', 'verification_status', 'purpose', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'tot_cur_bal', 'tot_coll_amt', 'total_rev_hi_lim',
       'mths_since_last_delinq', 'initial_list_status', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'acc_now_delinq',
       'term_months', 'bad_flag', 'emp_length_int',
       'mths_since_earliest_cr_line', 'mths_since_issue_d',
       'mths_since_last_pymnt_d', 'mths_since_next_pymnt_d',
       'mths_since_last_credit_pull_d'],
      dtype='object')

## numerical mising values

In [317]:
num_features = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment',
       'annual_inc','dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc','total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_cur_bal',
       'total_rev_hi_lim', 'term_months', 'bad_flag', 'emp_length_int',
       'mths_since_earliest_cr_line', 'mths_since_issue_d',
       'mths_since_last_pymnt_d', 'mths_since_next_pymnt_d',
       'mths_since_last_credit_pull_d']


for col in num_features:
    if load_data[col].isnull().mean()>0:
        print(col, round(load_data[col].isnull().mean(),4))

annual_inc 0.0
delinq_2yrs 0.0001
inq_last_6mths 0.0001
open_acc 0.0001
pub_rec 0.0001
revol_util 0.0007
total_acc 0.0001
collections_12_mths_ex_med 0.0003
acc_now_delinq 0.0001
tot_cur_bal 0.1507
total_rev_hi_lim 0.1507
emp_length_int 0.0451
mths_since_earliest_cr_line 0.0001
mths_since_last_pymnt_d 0.0008
mths_since_next_pymnt_d 0.4873
mths_since_last_credit_pull_d 0.0001


In [318]:
load_data['annual_inc'].fillna(load_data['annual_inc'].mean(), inplace=True)
load_data['mths_since_earliest_cr_line'].fillna(0, inplace=True)
load_data['acc_now_delinq'].fillna(0, inplace=True)
load_data['total_acc'].fillna(0, inplace=True)
load_data['pub_rec'].fillna(0, inplace=True)
load_data['open_acc'].fillna(0, inplace=True)
load_data['inq_last_6mths'].fillna(0, inplace=True)
load_data['delinq_2yrs'].fillna(0, inplace=True)
load_data['collections_12_mths_ex_med'].fillna(0, inplace=True)
load_data['revol_util'].fillna(0, inplace=True)
load_data['emp_length_int'].fillna(0, inplace=True)
load_data['tot_cur_bal'].fillna(0, inplace=True)
load_data['tot_coll_amt'].fillna(0, inplace=True)
load_data['mths_since_last_delinq'].fillna(-1, inplace=True)

## categorical missing values

In [319]:
cat_features = ['grade','verification_status','purpose','initial_list_status']

load_data[cat_features].isnull().mean()

grade                  0.0
verification_status    0.0
purpose                0.0
initial_list_status    0.0
dtype: float64

#Save load_data features

In [320]:
load_data.to_csv('/content/gdrive/MyDrive/dataset/load_data.csv', index = False)